<a href="https://colab.research.google.com/github/fatmabenhlel1/Few-Shot-Learning-/blob/main/2D_cnn_early_fusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import cv2
import random
import numpy as np
import datetime
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPool3D, Flatten, Dense, Dropout, Activation
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import GlobalAveragePooling3D, GlobalMaxPooling3D, Concatenate

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os

def count_folders(directory):
    return len([name for name in os.listdir(directory) if os.path.isdir(os.path.join(directory, name))])

# Exemple d'utilisation

directory3 = "/content/drive/MyDrive/Database/data"

print(f"Nombre de dossiers : {count_folders(directory3)}")


Nombre de dossiers : 7247


In [5]:
def extract_label(folder_name):
    """Extrait le label après le deuxième underscore."""
    parts = folder_name.split("_")
    if len(parts) >= 3:
        return int(parts[2])  # 0: Pt, 1: 72, 2: 1
    else:
        raise ValueError(f"Impossible d'extraire le label depuis : {folder_name}")


def create_txt_list(data_dir, output_txt):
    samples = []

    for video_folder in os.listdir(data_dir):
        label = extract_label(video_folder)
        video_path = os.path.join("data", video_folder)  # important : relatif à "data"
        samples.append(f"{video_path} 1 {label}\n")



    with open(output_txt, "w") as f:
        f.writelines(samples)

    print(f"Fichier généré avec {len(samples)} vidéos : {output_txt}")

# Usage
create_txt_list("/content/drive/MyDrive/Database/data", "all.txt")


Fichier généré avec 7247 vidéos : all.txt


In [6]:
def cnn2d_backbone():
    inputs = Input(shape=(112, 112, 3))
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)

    x_avg = GlobalAveragePooling2D()(x)
    x_max = GlobalMaxPooling2D()(x)

    features = Concatenate()([x_avg, x_max])  # final vector per frame
    model = Model(inputs, features)
    return model


In [7]:
from tensorflow.keras.models import Model

def video_embedding_model(input_dim=256, num_classes=4):
    inputs = tf.keras.Input(shape=(input_dim,))
    x = tf.keras.layers.Dense(128, activation='relu')(inputs)
    x = tf.keras.layers.Dropout(0.5)(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    return tf.keras.Model(inputs, outputs)



In [8]:
def generator_train_batch(train_file, batch_size, num_classes, img_path, cnn_model):
    with open(train_file, "r") as f:
        lines = f.readlines()

    while True:
        for i in range(0, len(lines), batch_size):
            batch_lines = lines[i:i + batch_size]
            X_batch = []
            y_batch = []

            for line in batch_lines:
                try:
                    parts = line.strip().split()
                    if len(parts) < 3:
                        print(f"Malformed line skipped: {line.strip()}")
                        continue

                    path = parts[0]
                    label = int(parts[2])  # 🟢 Assuming the label is the third element

                    folder_path = os.path.join(img_path, path)
                    frame_files = sorted(os.listdir(folder_path))

                    if len(frame_files) < 10:
                        print(f"Not enough frames in: {folder_path}")
                        continue

                    frames = []
                    for frame_file in frame_files[:10]:
                        img_path_full = os.path.join(folder_path, frame_file)
                        img = cv2.imread(img_path_full)

                        if img is None:
                            print(f"Could not read image: {img_path_full}")
                            raise ValueError("Image is None")

                        img = cv2.resize(img, (112, 112))
                        img = img / 255.0
                        frames.append(img)

                    frames = np.array(frames)  # shape (10, 112, 112, 3)

                    # 🟢 Extract features from each frame
                    features = cnn_model.predict(frames, verbose=0)  # (10, 256)
                    video_embedding = np.mean(features, axis=0)      # (256,)
                    X_batch.append(video_embedding)

                    # 🟢 One-hot encode the label
                    one_hot = tf.keras.utils.to_categorical(label, num_classes)
                    y_batch.append(one_hot)

                except Exception as e:
                    print(f"Error processing {line.strip()}: {e}")
                    continue

            if len(X_batch) == 0:
                continue

            yield np.array(X_batch), np.array(y_batch)


In [9]:
def load_frames_from_folder(folder_path):
    frames = []
    for frame_name in sorted(os.listdir(folder_path)):
        frame_path = os.path.join(folder_path, frame_name)
        img = cv2.imread(frame_path)
        img = cv2.resize(img, (112, 112))
        frames.append(img)
    frames = np.array(frames)  # (H, W, D, C) devient (H, D, W, C)
    return frames


In [10]:
class TimeHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        self.epoch_times = []

    def on_epoch_begin(self, epoch, logs=None):
        self.start_time = datetime.datetime.now()

    def on_epoch_end(self, epoch, logs=None):
        duration = (datetime.datetime.now() - self.start_time).total_seconds()
        self.epoch_times.append(duration)
        print(f" Temps pour l'epoch {epoch + 1}: {duration:.2f} secondes")


In [11]:
import os
import datetime
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf

def main():
    # === Chemins ===
    img_path = "/content/drive/MyDrive/Database/"  # correct path
    train_file = "all.txt"  # pas /train_list.txt

    # === Lire les fichiers pour compter les exemples ===
    with open(train_file, "r") as f:
        train_samples = len(f.readlines())

    # === Paramètres ===
    num_classes = 4
    batch_size = 8
    epochs = 25

    cnn_model = cnn2d_backbone()
    model = video_embedding_model(input_dim=cnn_model.output_shape[1], num_classes=num_classes)
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])


    model.summary()

    # === Création dossier checkpoints si nécessaire ===
    os.makedirs("checkpoints", exist_ok=True)

    # === TensorBoard ===
    logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

    # === Checkpoint Callback to save after each epoch ===
    checkpoint_callback = ModelCheckpoint(
        filepath="checkpoints/epoch_{epoch:02d}.h5",
        save_freq='epoch',
        save_weights_only=False,
        save_best_only=False,
        verbose=1
    )

    # === Time Callback ===
    time_callback = TimeHistory()




    # === Entraînement ===
    history = model.fit(
        generator_train_batch(train_file, batch_size, num_classes, img_path, cnn_model),
        steps_per_epoch=122,
        epochs=epochs,
        callbacks=[tensorboard_callback, checkpoint_callback, time_callback],
        verbose=1
    )
    # === Sauvegarde finale ===
    model.save("c3d_feature_extractor.h5")
    print("Feature extractor saved as c3d_feature_extractor.h5")




In [ ]:
if __name__ == "__main__":
    main()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 66,180 (258.52 KB)

 Trainable params: 66,180 (258.52 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/25
122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 53s/step - accuracy: 0.8751 - loss: 0.7664 
Epoch 1: saving model to checkpoints/epoch_01.h5


 Temps pour l'epoch 1: 6356.91 secondes
122/122 ━━━━━━━━━━━━━━━━━━━━ 6357s 53s/step - accuracy: 0.8741 - loss: 0.7685
Epoch 2/25
  2/122 ━━━━━━━━━━━━━━━━━━━━ 2:04:40 62s/step - accuracy: 0.1250 - loss: 0.9774Not enough frames in: /content/drive/MyDrive/Database/data/Pt_12_0_seq41
  5/122 ━━━━━━━━━━━━━━━━━━━━ 1:42:18 52s/step - accuracy: 0.1311 - loss: 1.0830

In [34]:
import os
import cv2
import numpy as np
import random
import tensorflow as tf
import traceback
from sklearn.metrics import classification_report
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import (
    Input,
    Conv2D,
    MaxPooling2D,
    GlobalAveragePooling2D,
    GlobalMaxPooling2D,
    Concatenate
)

# === Frame‐Level CNN Backbone ===
def cnn2d_backbone():
    inputs = Input(shape=(112, 112, 3))
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)

    x_avg = GlobalAveragePooling2D()(x)
    x_max = GlobalMaxPooling2D()(x)

    features = Concatenate(name='frame_features')([x_avg, x_max])  # 512D features
    return Model(inputs=inputs, outputs=features, name='cnn2d_backbone')


# === Load model with correct architecture ===
def load_backbone(model_path):
    """
    Try to load a full model and extract the frame_features layer. If that fails,
    rebuild the backbone and load weights by name.
    """
    try:
        full_model = load_model(model_path, compile=False)
        # Make sure our layer exists
        if 'frame_features' not in {l.name for l in full_model.layers}:
            raise ValueError("Layer 'frame_features' not found in loaded model.")
        return Model(
            inputs=full_model.input,
            outputs=full_model.get_layer('frame_features').output,
            name='backbone_from_full_model'
        )
    except Exception:
        print(f"[load_backbone] Could not load full model from {model_path}.")
        traceback.print_exc()
        print("[load_backbone] Falling back to rebuilding the backbone and loading weights by name...")
        backbone = cnn2d_backbone()
        try:
            backbone.load_weights(model_path, by_name=True)
            print("[load_backbone] Weights loaded into backbone successfully.")
            return backbone
        except Exception:
            print(f"[load_backbone] Failed to load weights from {model_path} into rebuilt backbone.")
            traceback.print_exc()
            raise


# === Load dataset ===
def load_dataset(dataset_dir, input_shape=(112, 112), max_frames=10):
    data = {}
    for class_name in sorted(os.listdir(dataset_dir)):
        class_path = os.path.join(dataset_dir, class_name)
        if not os.path.isdir(class_path):
            continue

        clips = []
        for clip_name in sorted(os.listdir(class_path)):
            clip_path = os.path.join(class_path, clip_name)
            frames = []
            for frame_name in sorted(os.listdir(clip_path))[:max_frames]:
                img = cv2.imread(os.path.join(clip_path, frame_name))
                if img is None:
                    continue
                img = cv2.resize(img, input_shape)
                frames.append(img.astype('float32') / 255.0)

            if len(frames) == max_frames:
                clips.append(np.stack(frames, axis=0))  # shape: (max_frames, H, W, C)

        # need at least 2 clips per class for few‐shot
        if len(clips) >= 2:
            data[class_name] = clips[:2]

    return data


# === Feature extraction ===
def extract_clip_feature(model, clip_frames):
    """
    clip_frames: np.array, shape (num_frames, H, W, C)
    returns: single 512‐D embedding vector
    """
    feats = model.predict(clip_frames, verbose=0)  # (num_frames, 512)
    return feats.mean(axis=0)


# === Few‐Shot Episodes ===
def run_episodes(data, model, num_episodes=1):
    y_true, y_pred = [], []
    class_names = list(data.keys())

    for ep in range(1, num_episodes + 1):
        support_feats, support_labels = [], []
        query_feats, query_labels = [], []

        # build support & query sets
        for idx, cname in enumerate(class_names):
            clips = data[cname]
            random.shuffle(clips)
            support, query = clips[:1][0], clips[1:2][0]

            support_feats.append(extract_clip_feature(model, support))
            support_labels.append(idx)

            query_feats.append(extract_clip_feature(model, query))
            query_labels.append(idx)

        # nearest‐neighbor classification
        for qf in query_feats:
            dists = [np.linalg.norm(qf - sf) for sf in support_feats]
            y_pred.append(int(np.argmin(dists)))
        y_true.extend(query_labels)

        print(f"Episode {ep}/{num_episodes} complete")

    return y_true, y_pred, class_names


# === Main ===
if __name__ == "__main__":
    model_path = "/content/checkpoints/epoch_01.h5"
    data_dir   = "/content/drive/MyDrive/Database/clips_fsl_v2"

    backbone = load_backbone(model_path)
    data     = load_dataset(data_dir)

    y_true, y_pred, classes = run_episodes(data, backbone)
    print("\nFew-Shot Classification Report:\n")
    print(classification_report(y_true, y_pred, target_names=classes))


[load_backbone] Could not load full model from /content/checkpoints/epoch_01.h5.
[load_backbone] Falling back to rebuilding the backbone and loading weights by name...
[load_backbone] Weights loaded into backbone successfully.


Traceback (most recent call last):
  File "<ipython-input-34-b0483e42e8b7>", line 47, in load_backbone
    raise ValueError("Layer 'frame_features' not found in loaded model.")
ValueError: Layer 'frame_features' not found in loaded model.


Episode 1/1 complete

Few-Shot Classification Report:

                     precision    recall  f1-score   support

          Carcinoma       1.00      1.00      1.00         1
   Extreme_polipoid       1.00      1.00      1.00         1
         Laryngitis       0.50      1.00      0.67         1
Vocal_insufficiency       0.00      0.00      0.00         1
        leukoplacia       1.00      1.00      1.00         1
          papilloma       1.00      1.00      1.00         1
               scar       1.00      1.00      1.00         1
    vocal_fold_cyst       0.00      0.00      0.00         1

           accuracy                           0.75         8
          macro avg       0.69      0.75      0.71         8
       weighted avg       0.69      0.75      0.71         8



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
